In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import * 
from pyspark.sql.window import *

In [0]:
# Define the data
data = [
 (1, "John", 30, "Sales", 50000.0),
 (2, "Alice", 28, "Marketing", 60000.0),
 (3, "Bob", 32, "Finance", 55000.0),
 (4, "Sarah", 29, "Sales", 52000.0),
 (5, "Mike", 31, "Finance", 58000.0)
]

In [0]:

# Define the schema
schema = StructType([
 StructField("id", IntegerType(), nullable=False),
 StructField("name", StringType(), nullable=False),
 StructField("age", IntegerType(), nullable=False),
 StructField("department", StringType(), nullable=False),
 StructField("salary", DoubleType(), nullable=False)
])

In [0]:
# Create the DataFrame
employeeDF = spark.createDataFrame(data, schema)

In [0]:
# Show the DataFrame
employeeDF.show()

+---+-----+---+----------+-------+
| id| name|age|department| salary|
+---+-----+---+----------+-------+
|  1| John| 30|     Sales|50000.0|
|  2|Alice| 28| Marketing|60000.0|
|  3|  Bob| 32|   Finance|55000.0|
|  4|Sarah| 29|     Sales|52000.0|
|  5| Mike| 31|   Finance|58000.0|
+---+-----+---+----------+-------+



##1. Calculate the average salary for each department

In [0]:
#Calculate the average salary for each department
avgSalbyDept=employeeDF\
    .groupBy("department")\
    .agg(avg("salary").alias("average_salary_by_dept"))

avgSalbyDept.display()

department,average_salary_by_dept
Sales,51000.0
Marketing,60000.0
Finance,56500.0


##2. Add a new column named "bonus" that is 10% of the salary for all employees.

In [0]:
#Add a new column named "bonus" that is 10% of the salary for all employees.
employeeDF=employeeDF.selectExpr('*','salary * 0.1 as bonus')
employeeDF.show()

+---+-----+---+----------+-------+------+
| id| name|age|department| salary| bonus|
+---+-----+---+----------+-------+------+
|  1| John| 30|     Sales|50000.0|5000.0|
|  2|Alice| 28| Marketing|60000.0|6000.0|
|  3|  Bob| 32|   Finance|55000.0|5500.0|
|  4|Sarah| 29|     Sales|52000.0|5200.0|
|  5| Mike| 31|   Finance|58000.0|5800.0|
+---+-----+---+----------+-------+------+



In [0]:
employeeDFbonus = employeeDF.withColumn("bonustest",col("salary")*lit(0.1))
employeeDFbonus.display()

id,name,age,department,salary,bonus,bonustest
1,John,30,Sales,50000.0,5000.0,5000.0
2,Alice,28,Marketing,60000.0,6000.0,6000.0
3,Bob,32,Finance,55000.0,5500.0,5500.0
4,Sarah,29,Sales,52000.0,5200.0,5200.0
5,Mike,31,Finance,58000.0,5800.0,5800.0


##3. Group the data by department and find the employee with the highest salary in each department

In [0]:
#Group the data by department and find the employee with the highest salary in each department
windowSpec =Window.partitionBy("department").orderBy(desc("salary"))
highestSalaryByDept=employeeDF.withColumn("row_number",row_number().over(windowSpec))
highestSalaryByDept.display()

id,name,age,department,salary,bonus,row_number
5,Mike,31,Finance,58000.0,5800.0,1
3,Bob,32,Finance,55000.0,5500.0,2
2,Alice,28,Marketing,60000.0,6000.0,1
4,Sarah,29,Sales,52000.0,5200.0,1
1,John,30,Sales,50000.0,5000.0,2


In [0]:
highestSalaryByDept.filter(col("row_number")==1).display()

id,name,age,department,salary,bonus,row_number
5,Mike,31,Finance,58000.0,5800.0,1
2,Alice,28,Marketing,60000.0,6000.0,1
4,Sarah,29,Sales,52000.0,5200.0,1


##4.  Find the top 3 departments with the highest total salary

In [0]:
#Find the top 3 departments with the highest total salary
highestTotalSalaryByDept=employeeDF\
    .groupBy("department")\
    .agg(sum("salary").alias("totalSalaryByDept"))\
    .orderBy(desc("totalSalaryByDept"))

highestTotalSalaryByDept.show(3)

+----------+-----------------+
|department|totalSalaryByDept|
+----------+-----------------+
|   Finance|         113000.0|
|     Sales|         102000.0|
| Marketing|          60000.0|
+----------+-----------------+



## 5. Find the top most department having highest salary

In [0]:
employeeDF.display()

id,name,age,department,salary,bonus
1,John,30,Sales,50000.0,5000.0
2,Alice,28,Marketing,60000.0,6000.0
3,Bob,32,Finance,55000.0,5500.0
4,Sarah,29,Sales,52000.0,5200.0
5,Mike,31,Finance,58000.0,5800.0


In [0]:
#Find the top most department having highest salary
windowSpecDept=Window.orderBy(desc("totalSalaryByDept"))
department_rnk = highestTotalSalaryByDept\
    .withColumn("rank",rank().over(windowSpecDept))

department_rnk.display()


department,totalSalaryByDept,rank
Finance,113000.0,1
Sales,102000.0,2
Marketing,60000.0,3


In [0]:
topdeptSalary=department_rnk.\
    filter(col("rank")==1)\
    .select("department")

topdeptSalary.display()

department
Finance


##6. Filter the DataFrame to keep only employees aged 30 or above and working in the "Sales" department

In [0]:
#Filter the DataFrame to keep only employees aged 30 or above and working in the "Sales" department
empFilteredDF= employeeDF.\
    filter((col("age")>=30) & (col("department")=="Sales"))

empFilteredDF.display()

id,name,age,department,salary,bonus
1,John,30,Sales,50000.0,5000.0


##7. Calculate the difference between each employee's salary and the average salary of their respective department

In [0]:
#Calculate the difference between each employee's salary and the average salary of their respective department
employeeDFavgSal =employeeDF.\
    withColumn("AvgSalarybyDept",avg(col("salary")).over(Window.partitionBy("department")))
employeeDFavgSal.display()

id,name,age,department,salary,bonus,AvgSalarybyDept
3,Bob,32,Finance,55000.0,5500.0,56500.0
5,Mike,31,Finance,58000.0,5800.0,56500.0
2,Alice,28,Marketing,60000.0,6000.0,60000.0
1,John,30,Sales,50000.0,5000.0,51000.0
4,Sarah,29,Sales,52000.0,5200.0,51000.0


In [0]:
empSalDiff=employeeDFavgSal.withColumn("salarydiff",col("salary")-col("AvgSalarybyDept"))
empSalDiff.display()

id,name,age,department,salary,bonus,AvgSalarybyDept,salarydiff
3,Bob,32,Finance,55000.0,5500.0,56500.0,-1500.0
5,Mike,31,Finance,58000.0,5800.0,56500.0,1500.0
2,Alice,28,Marketing,60000.0,6000.0,60000.0,0.0
1,John,30,Sales,50000.0,5000.0,51000.0,-1000.0
4,Sarah,29,Sales,52000.0,5200.0,51000.0,1000.0


##8. Calculate the sum of salaries for employees whose names start with the letter "J"

In [0]:
empFilteredName = employeeDF.filter(col("name").startswith("J")).\
  agg(sum("salary").alias("TotalSalary")
)

empFilteredName.display()

TotalSalary
50000.0


##9. Sort the DataFrame based on the "age" column in ascending order and then by "salary" column in descending order

In [0]:
empSortDF=employeeDF.sort("age",desc("salary"))

empSortDF.display()

id,name,age,department,salary,bonus
2,Alice,28,Marketing,60000.0,6000.0
4,Sarah,29,Sales,52000.0,5200.0
1,John,30,Sales,50000.0,5000.0
5,Mike,31,Finance,58000.0,5800.0
3,Bob,32,Finance,55000.0,5500.0


##10. Replace the department name "Finance" with "Financial Services" in the DataFrame

In [0]:
#Replace the department name "Finance" with "Financial Services" in the DataFrame

empDeptUpdated = employeeDF.withColumn("department",when(col("department")=="Finance","Financial Services").otherwise(col("department")))

empDeptUpdated.display()

id,name,age,department,salary,bonus
1,John,30,Sales,50000.0,5000.0
2,Alice,28,Marketing,60000.0,6000.0
3,Bob,32,Financial Services,55000.0,5500.0
4,Sarah,29,Sales,52000.0,5200.0
5,Mike,31,Financial Services,58000.0,5800.0


#11. Calculate the percentage of total salary each employee contributes to their respective department

In [0]:
#Calculate the percentage of total salary each employee contributes to their respective department
employeeDFTotalSalDept = employeeDF.withColumn("Total Salary",sum("salary").over(Window.partitionBy("department")))
employeeDFTotalSalDept.display()

id,name,age,department,salary,bonus,Total Salary
3,Bob,32,Finance,55000.0,5500.0,113000.0
5,Mike,31,Finance,58000.0,5800.0,113000.0
2,Alice,28,Marketing,60000.0,6000.0,60000.0
1,John,30,Sales,50000.0,5000.0,102000.0
4,Sarah,29,Sales,52000.0,5200.0,102000.0


In [0]:
employeeDFpercentSalDept = employeeDFTotalSalDept.withColumn(
    "Percentage_Contribution", round((col("salary") / col("Total Salary")) * 100, 2)
)

employeeDFpercentSalDept.display()

id,name,age,department,salary,bonus,Total Salary,Percentage_Contribution
3,Bob,32,Finance,55000.0,5500.0,113000.0,48.67
5,Mike,31,Finance,58000.0,5800.0,113000.0,51.33
2,Alice,28,Marketing,60000.0,6000.0,60000.0,100.0
1,John,30,Sales,50000.0,5000.0,102000.0,49.02
4,Sarah,29,Sales,52000.0,5200.0,102000.0,50.98
